In [22]:
pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 23.4 MB/s eta 0:00:00


In [2]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

# Make sure we don't get any GPU errors
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
(dataset_train,dataset_test), dataset_info =tfds.load (
    'mnist',
    split = ['train','test'],
    shuffle_files= True,
    as_supervised=True,  # will return tuple (img, label) otherwise dict
    with_info=True,  # able to get info about dataset
)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [5]:
dataset_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_dir='/root/tensorflow_datasets/mnist/3.0.1.incompleteQVJ7CB',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",

In [6]:
def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label

In [7]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

In [8]:
# Setup for train dataset
dataset_train = dataset_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
dataset_train = dataset_train.cache()
dataset_train = dataset_train.shuffle(dataset_info.splits["train"].num_examples)
dataset_train = dataset_train.batch(BATCH_SIZE)
dataset_train = dataset_train.prefetch(AUTOTUNE)

# Setup for test Dataset
dataset_test = dataset_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
dataset_test = dataset_test.batch(128)
dataset_test = dataset_test.prefetch(AUTOTUNE)

In [9]:
model = keras.Sequential(
    [
        keras.Input((28, 28, 1)),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Flatten(),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)

In [11]:
model.compile(
        optimizer =tf.keras.optimizers.Adam(learning_rate = 0.001),
        loss =keras.losses.SparseCategoricalCrossentropy(from_logits =False),
        metrics =['accuracy']
              )

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 flatten (Flatten)           (None, 21632)             0         
                                                                 
 dense (Dense)               (None, 10)                216330    
                                                                 
Total params: 216650 (846.29 KB)
Trainable params: 216650 (846.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
model.fit (dataset_train, epochs=10, verbose =2 )


Epoch 1/10
469/469 - 7s - loss: 0.2758 - accuracy: 0.9222 - 7s/epoch - 14ms/step
Epoch 2/10
469/469 - 2s - loss: 0.1011 - accuracy: 0.9723 - 2s/epoch - 3ms/step
Epoch 3/10
469/469 - 2s - loss: 0.0709 - accuracy: 0.9800 - 2s/epoch - 3ms/step
Epoch 4/10
469/469 - 1s - loss: 0.0552 - accuracy: 0.9832 - 1s/epoch - 3ms/step
Epoch 5/10
469/469 - 1s - loss: 0.0445 - accuracy: 0.9868 - 1s/epoch - 3ms/step
Epoch 6/10
469/469 - 1s - loss: 0.0369 - accuracy: 0.9898 - 1s/epoch - 3ms/step
Epoch 7/10
469/469 - 1s - loss: 0.0315 - accuracy: 0.9909 - 1s/epoch - 3ms/step
Epoch 8/10
469/469 - 1s - loss: 0.0269 - accuracy: 0.9920 - 1s/epoch - 3ms/step
Epoch 9/10
469/469 - 1s - loss: 0.0226 - accuracy: 0.9934 - 1s/epoch - 3ms/step
Epoch 10/10
469/469 - 1s - loss: 0.0183 - accuracy: 0.9949 - 1s/epoch - 3ms/step


In [14]:
model.evaluate(dataset_test, verbose =2 )

79/79 - 1s - loss: 0.0665 - accuracy: 0.9809 - 916ms/epoch - 12ms/step


[0.06651551276445389, 0.98089998960495]